# **Data processing**

### We took the data from zolo website. Checked the details of all the columns.
### Now a few of the blocks were required to be changed, this comes under cleaning of data. Hence, we cleaned the noise in the data 
### We checked the unique values of all the columns and then made changes that were required.
### Plotted the scatter plots between different columns which shows the dependencies of the variables on each other. Also we can get to know the residuals or any outliers.  
### Also, Plotted histogram, boxplot and subplots. Where histogram shows the rise and fall of the value. boxplots tells min and max values. 
### Also did we plotted the heat map that shows the co relation between the variables. 

###We checked all the available columns in the data and then checked for unique values for each and made any changes required for instance in walk score there was a ‘-‘  which was removed latersince its a noisy data. From scatterplot we conclude that with the greater number of bathrooms there are greater number of bedrooms as well. Also did we found that there are heavy number of walk score in larger size(sq feet). But this not true in case of rooms and stories that is if room number increases then stories are not increasing also there are not many homes with a lot of rooms. Also we found that the price of the house is highly dependent on the number of bathrooms. After that we performed linear regression using sklearn and found the co efficient of determinations.


In [ ]:
#import required libaries
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

In [ ]:
#picking the file to read from the drive 
drive_url = 'https://docs.google.com/spreadsheets/d/1IcfryaWf6HHAwQ_elcq5L1BCULhIINgR/edit#gid=1524578348' #Read the Excel Sheet, which contains 4000 House properties scraped from zolo.ca.
new_url = 'https://drive.google.com/uc?export=download&id=' + drive_url.split('/')[-2] #Save into temporary location
df = pd.read_excel(new_url)
df.head()

,Type,Walk Score,Age,Listed By,Size (sq ft),Bedrooms,Bathrooms,Kitchens,Rooms,Area,Den/Family Room,Patio Terrace,Ensuite Laundry,Air Conditioning,Fireplace,Stories,Parking Total,Maintenance,Taxes,Community,Basement,house_price
0,Condo Townhouse,83,No Data,"Royal Lepage Supreme Realty, Brokerage",1400-1599,3,3,1,8,Toronto,N,Terr,Y,Central Air,N,2,1.0,446.40,3250.59,Dovercourt-Wallace Emerson-Junction,None,999900
1,Condo Apt,37,No Data,"Royal Lepage Terrequity Realty, Brokerage",900-999,2,1,1,6,Toronto,N,Open,Y,Central Air,N,15,1.0,697.42,1255.6,Flemingdon Park,None,525000
2,Condo Apt,97,No Data,"Homelife Frontier Realty Inc., Brokerage",600-699,1,1,1,5,Toronto,N,Open,Y,Central Air,N,19,0.0,435.14,2480.71,Church-Yonge Corridor,None,499999
3,Condo Townhouse,30,No Data,"Re/max Millennium Real Estate, Brokerage",1000-1199,3,2,1,5,Toronto,N,Terr,Y,None,Y,1,2.0,649.93,1438.29,Elms-Old Rexdale,Crawl Space,529800
4,Detached,37,No Data,"First Class Realty Inc., Brokerage",3500-5000,4,6,1,10,Toronto,Y,NO,N,Central Air,Y,0,4.0,0.00,15617,St. Andrew-Windfields,Walk-Up,4280000


In [ ]:
#to check how many null values are there 
df.isnull().sum()
# size = 861 null values

Type                  0
Walk Score            0
Age                  11
Listed By             0
Size (sq ft)        861
Bedrooms              0
Bathrooms             0
Kitchens              0
Rooms                 0
Area                 12
Den/Family Room       0
Patio Terrace         0
Ensuite Laundry       0
Air Conditioning      0
Fireplace             0
Stories               1
Parking Total         0
Maintenance           0
Taxes                 0
Community            23
Basement              0
house_price           0
dtype: int64

In [ ]:
#getting the information of all the columns 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3917 entries, 0 to 3916
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Type              3917 non-null   object 
 1   Walk Score        3917 non-null   object 
 2   Age               3906 non-null   object 
 3   Listed By         3917 non-null   object 
 4   Size (sq ft)      3056 non-null   object 
 5   Bedrooms          3917 non-null   int64  
 6   Bathrooms         3917 non-null   int64  
 7   Kitchens          3917 non-null   int64  
 8   Rooms             3917 non-null   int64  
 9   Area              3905 non-null   object 
 10  Den/Family Room   3917 non-null   object 
 11  Patio Terrace     3917 non-null   object 
 12  Ensuite Laundry   3917 non-null   object 
 13  Air Conditioning  3917 non-null   object 
 14  Fireplace         3917 non-null   object 
 15  Stories           3916 non-null   object 
 16  Parking Total     3917 non-null   float64


In [ ]:
# checking the unique values available in 'type' column
df['Type'].nunique(),df['Type'].unique()

# Some unique values are merged as they are having the same meaning
# "condoxxxxxx" -> condo
# co-op apt, Apartments, Co-Ownership Apt -> Apartments
# twnhouse -> town house
# triplex, multiplex, duplex, Fourplex -> multiplex
# Single Family,House/Single Family-> Single Family
# Link, Parking Space, Locker, Parking, Land, Vacant Land -> ohters
# retail, Commercial/Retail -> retail

(33, array(['Condo Townhouse', 'Condo Apt', 'Detached', 'Att/Row/Twnhouse',
        'Semi-Detached', 'Comm Element Condo', 'Co-Op Apt', 'Investment',
        'Det Condo', 'Vacant Land', 'Apartments', 'Condominium', 'Triplex',
        'Link', 'Multiplex', 'Condo/Apt Unit', 'Other', 'Co-Ownership Apt',
        'Duplex', 'Parking Space', 'Locker', 'Single Family',
        'Commercial/Retail', 'Industrial', 'Land', 'Office',
        'Row / Townhouse', 'Leasehold Condo', 'Fourplex',
        'House/Single Family', 'Retail', 'Store W/Apt/Office', 'Parking'],
       dtype=object))

In [ ]:
# processing the 'Type' feature

def process_type(house_type):
  if 'condo' in house_type.lower():
    return 'condo'
  if 'apt' in house_type.lower():
    return 'Apartments'
  if house_type.lower() in ['duplex','triplex','fourplex','multiplex']:
    return 'multiplex'
  if house_type in ['Single Family','House/Single Family']:
    return 'single_family'
  if house_type in ['Link','Parking Space','Locker','Parking','Vacant Land','Land']:
    return 'Other'
  if 'retail' in house_type.lower():
    return 'retail'
  if house_type in ['Att/Row/Twnhouse','Row / Townhouse']:
    return 'townhouse'
  return house_type

df['Type'] = df['Type'].apply(process_type)

In [ ]:
#getting all the unique types of 'Type' column
df['Type'].unique()

array(['condo', 'Detached', 'townhouse', 'Semi-Detached', 'Apartments',
       'Investment', 'Other', 'multiplex', 'single_family', 'retail',
       'Industrial', 'Office'], dtype=object)

In [ ]:
# Counting all the unique types
df['Type'].value_counts()

condo            2594
Detached          816
Semi-Detached     180
Other             100
townhouse          89
multiplex          48
Apartments         47
Investment         26
Office              6
retail              5
single_family       5
Industrial          1
Name: Type, dtype: int64

In [ ]:
df['Walk Score'].unique()

array([83, 37, 97, 30, 70, 79, 77, 38, 48, 82, 91, 98, 72, 40, 31, 90, 57,
       80, 18, 66, 76, 68, 99, 67, 61, 85, 50, 74, 59, 92, 84, 60, 39, 53,
       45, 6, 87, 89, 32, 88, 81, 46, 63, 51, 23, 34, 78, 75, 41, 58, 93,
       94, 17, 86, 65, 33, 47, 28, 15, 62, 25, 49, 35, 3, 29, 12, 56, 42,
       10, 69, 95, 73, 16, 71, 96, 54, 44, 52, 20, 14, 22, 19, 27, 43, 55,
       64, 8, 24, 26, 9, 36, 11, '–', 21, 7, 13, 4], dtype=object)

In [ ]:
#removing the '-' from walk score 
temp_df = df.copy()
temp_df['Walk Score'] = pd.to_numeric(temp_df.where(temp_df['Walk Score'] != '–')['Walk Score'])
temp_df = temp_df.groupby('Community')[['Walk Score','Bedrooms']].mean().astype('int')

for i in range(0,len(df.index)):
  if df.iloc[i]['Walk Score'] == '–':
    df.loc[i, 'Walk Score'] =  temp_df.loc[df.iloc[i]['Community']]['Walk Score']

df.drop(labels=['Community'],axis=1,inplace=True) #We no longer need the 'Community' column because we already have a 'Walk Score' that we defined from the 'Community'.

In [ ]:
df['Walk Score'] = df['Walk Score'].astype(int)
df['Walk Score'].unique()

array([83, 37, 97, 30, 70, 79, 77, 38, 48, 82, 91, 98, 72, 40, 31, 90, 57,
       80, 18, 66, 76, 68, 99, 67, 61, 85, 50, 74, 59, 92, 84, 60, 39, 53,
       45,  6, 87, 89, 32, 88, 81, 46, 63, 51, 23, 34, 78, 75, 41, 58, 93,
       94, 17, 86, 65, 33, 47, 28, 15, 62, 25, 49, 35,  3, 29, 12, 56, 42,
       10, 69, 95, 73, 16, 71, 96, 54, 44, 52, 20, 14, 22, 19, 27, 43, 55,
       64,  8, 24, 26,  9, 36, 11, 21,  7, 13,  4])

In [ ]:
#We're removing it since it has too many distinct values can't provide much information towards the house price
print(df['Listed By'].nunique()) 

#We're removing it since has too many null values
(df['Age'] == 'No Data').value_counts()

543


True     2454
False    1463
Name: Age, dtype: int64

In [ ]:
# dropping ['Listed By','Age'] column
df.drop(labels=['Listed By','Age'],axis=1,inplace=True)

In [ ]:
# unique values in bedroom column
df['Bedrooms'].unique()

array([3, 2, 1, 4, 5, 0, 7, 9, 6, 8])

In [ ]:
# Converting the bathroom column into integer type
df['Bathrooms'] = df['Bathrooms'].astype(int)

# unique values in bathroom column
df['Bathrooms'].unique()

array([ 3,  1,  2,  6,  4,  5,  9,  0,  7, 10,  8, 14, 12, 16])

In [ ]:
# unique values in Kitchens column
df['Kitchens'].unique()

array([1, 2, 0, 3, 5, 4, 6, 9])

In [ ]:
# unique values in Rooms column
df['Rooms'].unique()

array([ 8,  6,  5, 10,  4,  9,  7, 11,  3,  0, 14, 12, 13,  2,  1, 18, 15,
       16, 19, 20, 48, 24, 23, 21, 30, 33, 17, 25])

In [ ]:
# Drop properties with zeros in all four parameters (Bedrooms, Bathrooms, Kitchens, and Rooms).
df.drop(df[(df['Bedrooms'] == 0) & (df['Bathrooms'] == 0) & (df['Kitchens'] == 0) & (df['Rooms'] == 0)].index,inplace=True) 

In [ ]:
#Because all of the values are the same, the area column is being removed.
print(df['Area'].unique()) 
df.drop(labels=['Area'],axis=1,inplace=True)

['Toronto' nan]


In [ ]:
# unique values in Den/Family Rooms column
df['Den/Family Room'].unique() 

array(['N', 'Y', 0], dtype=object)

In [ ]:
# For all family rooms, change 0 to N.
def process_family_room(family_room):
  if family_room == 0:
    return 'N'
  else:
    return family_room

df['Den/Family Room'] = df['Den/Family Room'].apply(process_family_room)
df['Den/Family Room'].unique()

array(['N', 'Y'], dtype=object)

In [ ]:
# unique values in Patio Terrace column
df['Patio Terrace'].unique()

array(['Terr', 'Open', 'NO', 'None', 'Jlte', 'Encl', 4], dtype=object)

In [ ]:
# Change followingg values..
# 'NO', 'None', 'Jlte', 'Encl', '4' --> 'No'
# 'Terr', 'Open' --> 'Yes'
def process_patio(terrace):
  if terrace in ['NO', 'None', 'Jlte', 'Encl', 4]:
    return 'No'
  if terrace in ['Open', 'Terr']:
    return 'Yes'
  return terrace

df['Patio Terrace'] = df['Patio Terrace'].apply(process_patio)
df['Patio Terrace'].unique()

array(['Yes', 'No'], dtype=object)

In [ ]:
# unique values in 'Ensuite Laundry' column
df['Ensuite Laundry'].unique()

array(['Y', 'N'], dtype=object)

In [ ]:
# unique values in 'Air Conditioning' column
df['Air Conditioning'].unique()

array(['Central Air', 'None', 'Window Unit', 'Wall Unit', 'Other',
       'def no data', 'Central Air Conditioning', 'Part', 'Y', 'N'],
      dtype=object)

In [ ]:
# ['Central Air','Window Unit', 'Wall Unit', 'Y', 'Central Air Conditioning', 'Part'] is replaced by 'Y', remaining everything with no
def process_conditioning(air_conditioning):
  if air_conditioning in ['Central Air','Window Unit', 'Wall Unit', 'Y', 'Central Air Conditioning', 'Part']:
    return 'Y'
  else:
    return 'N' 

df['Air Conditioning'] = df['Air Conditioning'].apply(process_conditioning)
df['Air Conditioning'].unique()

array(['Y', 'N'], dtype=object)

In [ ]:
# unique values in 'Fireplace' column
df['Fireplace'].unique() 

array(['N', 'Y', False], dtype=object)

In [ ]:
#Change 'False' to 'N'
def process_fireplace(s):
  if s == 'Y':
    return 'Y'
  else:
    return 'N' 

df['Fireplace'] = df['Fireplace'].apply(process_fireplace)
df['Fireplace'].unique()

array(['N', 'Y'], dtype=object)

In [ ]:
# unique values in 'Stories' column
df['Stories'].unique()

array([2, 15, 19, 1, 0, 20, 9, 10, 14, 16, 18, 6, 24, 4, 52, 8, 17, 11, 7,
       34, 3, 37, 27, 13, 12, 5, 28, 21, 44, 42, 26, 30, 32, 74, 25, 40,
       23, 36, 22, 29, 38, 50, 31, 39, 46, 'Ph', 64, 35, '0-1', 45, 'Lph',
       33, '1&2', 51, 47, 'P1', 'A', 54, 48, 65, 59, 41, 53, 56, 49, 'M',
       79, 58, 69, 60, 57, 'L15', 'Low', '3&4', 43, '1st', 'Th'],
      dtype=object)

In [ ]:
# replacing values with below list
zero = ['Ph','Lph','Th']
one = ['P1','0-1','1&2','A','Low','1st','L15','M']
three = ['3&4']

def process_stories(stroy):
  if stroy in zero:
    return 0
  elif stroy in one:
    return 1
  elif stroy in three:
    return 3
  else:
    return stroy

df['Stories'] = df['Stories'].apply(process_stories)
df['Stories'].unique() 

array([ 2, 15, 19,  1,  0, 20,  9, 10, 14, 16, 18,  6, 24,  4, 52,  8, 17,
       11,  7, 34,  3, 37, 27, 13, 12,  5, 28, 21, 44, 42, 26, 30, 32, 74,
       25, 40, 23, 36, 22, 29, 38, 50, 31, 39, 46, 64, 35, 45, 33, 51, 47,
       54, 48, 65, 59, 41, 53, 56, 49, 79, 58, 69, 60, 57, 43])

In [ ]:
# unique values in 'Parking Total' column
df['Parking Total'].unique() 

array([  1. ,   0. ,   2. ,   4. ,   3. ,   6. ,   5. ,   9. ,  12. ,
         8. ,   7. ,  10. ,   1.5,   5.5, 106. ,  23. ,  14. ,  33. ,
        16. ,  13. ,  18. ,   3.5,   2.5,  22. ,  11. ,  24. ,  15. ,
        17. ,  25. ,  20. ])

In [ ]:
# Converting values into integer from the float
df['Parking Total'] = df['Parking Total'].astype(int)

In [ ]:
# number of rows having 'No Data' 
(df['Taxes'] == 'No Data').value_counts()

False    3772
True       20
Name: Taxes, dtype: int64

In [ ]:
# no data = 0 for the rows having 'no data' 
def process_taxes(taxes):
  if taxes == 'No Data':
    return 0
  else:
    return taxes

df['Taxes'] = df['Taxes'].apply(process_taxes)
df['Taxes'] = df['Taxes'].astype(float)
df[['Taxes']].describe()

,Taxes
count,3792.000000
mean,4375.948747
std,9101.676632
min,0.000000
25%,1729.160000
50%,2841.210000
75%,4900.000000
max,333002.000000


In [ ]:
# unique values in 'Basement' column
df['Basement'].unique()

array(['None', 'Crawl Space', 'Walk-Up', 'Sep Entrance', 'Full',
       'Finished', 'W/O', 'Unfinished', 'Part Bsmt', 'Part Fin',
       'Fin W/O', 'Apartment', 'Other', 'Half', 'Y', 'No', 'Yes',
       'Partial (Finished)', 'Full (Unfinished)'], dtype=object)

In [ ]:
# changing below list values to 'yes' and others to 'no'
yes = ['Crawl Space', 'Walk-Up', 'Sep Entrance', 'Full',
       'Finished', 'W/O', 'Unfinished', 'Part Bsmt', 'Part Fin',
       'Fin W/O', 'Y','Yes','Partial (Finished)', 'Full (Unfinished)']

# processing basement column 
def process_basement(basement):
  if basement in yes:
    return 'Y'
  else:
    return 'N'

df['Basement'] = df['Basement'].apply(process_basement)
df['Basement'].unique()

array(['N', 'Y'], dtype=object)

In [ ]:
# values in the 'size' columns
df['Size (sq ft)'].unique()

array(['1400-1599', '900-999', '600-699', '1000-1199', '3500-5000',
       '500-599', '700-799', nan, '1500-2000', '2000-2500', '1100-1500',
       '800-899', '2500-3000', '0-499', '2000-2249', '1600-1799',
       '1200-1399', '1800-1999', '3000-3500', '2250-2499', '700-1100',
       '5000+', '3250-3499', '3000-3249', '2750-2999', '< 700',
       '2500-2749', '4000-4249', '4750-4999', '900-1099', '1100-1299',
       '3750-3999', '3500-3749', '4250-4499', '4500-4749'], dtype=object)

In [ ]:
# processing size feature 
def process_size(house_size):
  if type(house_size) == str :
    if '-' in house_size:
      temp = house_size.split('-')
      return (int(temp[0])+int(temp[1]))/2
    elif '<' in house_size:
      temp = house_size.split('<')
      return int(temp[1])
    elif '+' in house_size:
      temp = house_size.split('+')
      return int(temp[0])
    else:
      return int(house_size) 

df['Size (sq ft)'] = df['Size (sq ft)'].apply(process_size)

In [ ]:
df.isnull().sum()

Type                  0
Walk Score            0
Size (sq ft)        752
Bedrooms              0
Bathrooms             0
Kitchens              0
Rooms                 0
Den/Family Room       0
Patio Terrace         0
Ensuite Laundry       0
Air Conditioning      0
Fireplace             0
Stories               0
Parking Total         0
Maintenance           0
Taxes                 0
Basement              0
house_price           0
dtype: int64

In [ ]:
# now we fill values for 'size (sq ft)' using linear regression with ['Bedrooms','Bathrooms','Kitchens','Rooms']

In [ ]:
# making new dataframe using the feaures that we use for the predicting null values of the target ('Size (sq ft)'0)
df_new = df[['Bedrooms','Bathrooms','Kitchens','Rooms','Size (sq ft)']]

In [ ]:
# dropping all the null values for now
df_new.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# finding the correlation between features and target
df_new.corr()

,Bedrooms,Bathrooms,Kitchens,Rooms,Size (sq ft)
Bedrooms,1.000000,0.780225,0.327940,0.776677,0.775396
Bathrooms,0.780225,1.000000,0.259881,0.754701,0.875385
Kitchens,0.327940,0.259881,1.000000,0.408507,0.299488
Rooms,0.776677,0.754701,0.408507,1.000000,0.777864
Size (sq ft),0.775396,0.875385,0.299488,0.777864,1.000000


In [ ]:
df['Size (sq ft)'].isnull().sum()

752

In [ ]:
# separating features and the target columns
X =  df_new[['Bedrooms','Bathrooms','Kitchens','Rooms']]
y = df_new['Size (sq ft)']

# Splitting the data into train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=11)

In [ ]:
# Importing linear regression model from sklearn's linear model library
from sklearn.linear_model import LinearRegression

# Taking a linear regression instace
regression_model = LinearRegression()

# Fitting the data to model
regression_model.fit(X_train.values,y_train.values)

LinearRegression()

In [ ]:
# constant/intercept for our equation
regression_model.intercept_

-467.55117708927514

In [ ]:
# Predicting model output on the unseen data
y_pred = regression_model.predict(X_test.values)

In [ ]:
# importing metrics for evaluation
from sklearn import metrics

# Finding R2 score, which is also known as coefficient of determination
print('Coefficient of determination:', metrics.r2_score(y_test, y_pred))

Coefficient of determination: 0.803875709641004


In [ ]:
# Coefficients for our linear regression equation to see which how much each varible is contributing
print(regression_model.coef_)

[118.10845404 453.75741183  59.71446816  84.83121519]


In [ ]:
# Now we will use the machine learning model to predict the values for the null values present in the size column

# First, let's find out index of the null values
null_index_list = df[df['Size (sq ft)'].isnull()].index.tolist()

for idx in null_index_list:
  
  # getting the feature values to predict the target
  temp = df.loc[idx,['Bedrooms','Bathrooms','Kitchens','Rooms']]
  Bedrooms = temp['Bedrooms']
  Bathrooms = temp['Bathrooms'] 
  Kitchens = temp['Kitchens']
  Rooms = temp['Rooms']

  # Making the prediction and replacing null values with it
  df.loc[idx,['Size (sq ft)']] = regression_model.predict([[Bedrooms,Bathrooms,Kitchens,Rooms]])

In [ ]:
df['Size (sq ft)'].isnull().sum()

0

In [ ]:
# checking null values if there is any
df.isnull().sum()

Type                0
Walk Score          0
Size (sq ft)        0
Bedrooms            0
Bathrooms           0
Kitchens            0
Rooms               0
Den/Family Room     0
Patio Terrace       0
Ensuite Laundry     0
Air Conditioning    0
Fireplace           0
Stories             0
Parking Total       0
Maintenance         0
Taxes               0
Basement            0
house_price         0
dtype: int64